<a href="https://colab.research.google.com/github/YangyangFu/transformer-time-series/blob/main/examples/train_tide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import os
import random
import string
import sys

from absl import app
from absl import flags
from absl import logging
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm

2023-09-14 10:08:44.513267: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-14 10:08:44.536771: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-14 10:08:44.864588: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Install Dependency

In [2]:
!pip install git+https://github.com/YangyangFu/transformer-time-series@main

/bin/bash: /home/yyf/miniconda3/envs/tts-tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Cloning https://github.com/YangyangFu/transformer-time-series (to revision main) to /tmp/pip-req-build-pcrx6ao2
  Running command git clone --filter=blob:none --quiet https://github.com/YangyangFu/transformer-time-series /tmp/pip-req-build-pcrx6ao2
  Resolved https://github.com/YangyangFu/transformer-time-series to commit 64f88f790876344f9edca3c080d51cc2835a9fe4
  Preparing metadata (setup.py) ... done


In [3]:
from tsl.dataloader.batch_on_ts import DataLoader
from tsl.tide import TIDE
from tsl.utils.utils import seed_everything

seed_everything(1024)


Random seed set as 1024


## Download Dataset

In [ ]:
!apt install subversion
!svn checkout https://github.com/YangyangFu/transformer-time-series/trunk/datasets-raw
!cd ./datasets-raw && bash download_data.sh

Comment the following code out if not needed.

In [7]:
!cd ./datasets-raw && python process_ETTh1.py

/bin/bash: /home/yyf/miniconda3/envs/tts-tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: line 0: cd: ./datasets-raw: No such file or directory


## Metrics and Functions


In [4]:
EPS=1E-06


def _get_random_string(num_chars):
  rand_str = ''.join(
      random.choice(
          string.ascii_uppercase + string.ascii_lowercase + string.digits
      )
      for _ in range(num_chars - 1)
  )
  return rand_str


 # metrics 
def mape(y_pred, y_true):
  abs_diff = np.abs(y_pred - y_true).flatten()
  abs_val = np.abs(y_true).flatten()
  idx = np.where(abs_val > EPS)
  mpe = np.mean(abs_diff[idx] / abs_val[idx])
  return mpe


def mae_loss(y_pred, y_true):
  return np.abs(y_pred - y_true).mean()


def wape(y_pred, y_true):
  abs_diff = np.abs(y_pred - y_true)
  abs_val = np.abs(y_true)
  wpe = np.sum(abs_diff) / (np.sum(abs_val) + EPS)
  return wpe


def smape(y_pred, y_true):
  abs_diff = np.abs(y_pred - y_true)
  abs_mean = (np.abs(y_true) + np.abs(y_pred)) / 2
  smpe = np.mean(abs_diff / (abs_mean + EPS))
  return smpe


def rmse(y_pred, y_true):
  return np.sqrt(np.square(y_pred - y_true).mean())


def nrmse(y_pred, y_true):
  mse = np.square(y_pred - y_true)
  return np.sqrt(mse.mean()) / np.abs(y_true).mean()


METRICS = {
    'mape': mape,
    'wape': wape,
    'smape': smape,
    'nrmse': nrmse,
    'rmse': rmse,
    'mae': mae_loss,
}


## Experiment Settings

In [5]:
"""Training TS code."""
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
#tf.config.experimental.set_visible_devices([], 'GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

experiment_id = _get_random_string(8)
logging.info('Experiment id: %s', experiment_id)



2023-09-14 10:09:19.498871: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-14 10:09:19.939913: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-14 10:09:19.940341: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

## Create Dataloader

In [9]:
# model settings
source_seq_len = 720
pred_len = 96
target_seq_len = pred_len
target_cols=['HUFL','HULL','MUFL','MULL','LUFL','LULL','OT']

# get data path
data_path = "./datasets/ETT-small/ETTh1"

ts_file = "ts.joblib"
cat_cov_local_invariant_file='id.joblib'

# create dataloader
dataloader = DataLoader(
        data_path=data_path,
        ts_file=ts_file,
        num_cov_global_file=None,
        cat_cov_global_file=None,
        num_cov_local_variant_file=[],
        cat_cov_local_variant_file=[],
        num_cov_local_invariant_file=[],
        cat_cov_local_invariant_file=cat_cov_local_invariant_file,
        num_cov_local_variant_names=[],
        cat_cov_local_variant_names=[],
        target_cols=target_cols,
        train_range=(0, 24*30*12),
        val_range=(24*30*12, 24*30*16),
        test_range=(24*30*16, 24*30*20),
        hist_len=source_seq_len,
        token_len=target_seq_len-pred_len,
        pred_len=pred_len,
        batch_size=min(32, len(target_cols)),
        freq='H',
        normalize=True,
        use_time_features=True,
        use_holiday=True,
        use_holiday_distance=True,
        normalize_time_features=True,
        use_history_for_covariates=True
)

train_ds = dataloader.generate_dataset(mode="train", shuffle=True, seed=1)
val_ds = dataloader.generate_dataset(mode="validation", shuffle=False, seed=1)
test_ds = dataloader.generate_dataset(mode="test", shuffle=False, seed=1)


                         HUFL      HULL      MUFL      MULL      LUFL  \
date                                                                    
2016-07-01 00:00:00  5.828125  2.009766  1.598633  0.461914  4.203125   
2016-07-01 01:00:00  5.691406  2.076172  1.492188  0.426025  4.140625   
2016-07-01 02:00:00  5.156250  1.741211  1.279297  0.354980  3.777344   
2016-07-01 03:00:00  5.089844  1.942383  1.279297  0.391113  3.806641   
2016-07-01 04:00:00  5.359375  1.942383  1.492188  0.461914  3.867188   

                         LULL         OT  
date                                      
2016-07-01 00:00:00  1.339844  30.531250  
2016-07-01 01:00:00  1.371094  27.781250  
2016-07-01 02:00:00  1.217773  27.781250  
2016-07-01 03:00:00  1.279297  25.046875  
2016-07-01 04:00:00  1.279297  21.953125  
Generating time features.................
       HUFL      HULL      MUFL      MULL      LUFL      LULL        OT
0 -0.362793 -0.005398 -0.630859 -0.147461  1.388672  0.874512  1.459961
1

2023-09-14 10:11:25.644521: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-14 10:11:25.644646: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-14 10:11:25.644695: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [10]:
# create model
hidden_size = 512
num_layers = 2
decoder_output_dim = 32
hidden_dims_time_decoder = 16
layer_norm = True 
dropout_rate = 0.5

model = TIDE(
    pred_length=pred_len,
    hidden_dims_encoder=[hidden_size] * num_layers, 
    output_dims_encoder=[hidden_size] * num_layers, 
    hidden_dims_decoder=[hidden_size], 
    output_dims_decoder=[decoder_output_dim*pred_len], 
    hidden_dims_time_encoder=64,
    output_dims_time_encoder=4,
    hidden_dims_time_decoder=hidden_dims_time_decoder,
    cat_sizes=[1],# a fake 0 for categorical
    cat_emb_size=4,
    num_ts=len(target_cols),
    layer_norm=layer_norm,
    dropout_rate=dropout_rate,
)

In [11]:
# LR scheduling
learning_rate = 0.000984894211777642
lr_schedule = keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=learning_rate,
    decay_steps=30 * dataloader.train_range[1],
)

# loss function
loss_fcn = keras.losses.MeanSquaredError()
optimizer = keras.optimizers.Adam(learning_rate=lr_schedule, clipvalue=1e3)

train_metrics = [tf.keras.metrics.MeanAbsoluteError()]
val_metrics = [tf.keras.metrics.MeanAbsoluteError()]
test_metrics = [tf.keras.metrics.MeanSquaredError(), tf.keras.metrics.MeanAbsoluteError()]

# train step
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        y_pred = model(x, training=True)
        loss = loss_fcn(y, y_pred)
    
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    # update metrics
    for metric in train_metrics:
            metric.update_state(y, y_pred)
    return loss

# validation step
@tf.function
def val_step(x, y):
    y_pred = model(x, training=False)
    loss = loss_fcn(y, y_pred)
    for metric in val_metrics:
        metric.update_state(y, y_pred)
    return loss

# test step
@tf.function
def test_step(x, y):
    y_pred = model(x, training=False)
    for metric in test_metrics:
        metric.update_state(y, y_pred)



In [12]:
# main loop
MAX_EPOCHS = 100
patience = 20
wait = 0
best_val_loss = np.inf

for epoch in range(MAX_EPOCHS):
    # take a batch
    for batch in train_ds:
        enc, dec = batch
        (ts_enc, num_global_enc, cat_global_enc, 
         num_local_variant_enc, cat_local_variant_enc, 
         num_local_invariant_enc, cat_local_invariant_enc, time_features_enc) = enc
        (ts_dec, num_global_dec, cat_global_dec, 
         num_local_variant_dec, cat_local_variant_dec, 
         num_local_invariant_dec, cat_local_invariant_dec, time_features_dec) = dec
        
        # Note the orginal paper uses constant 0 for categorical features if not specified.
        # (B, L), (nx, L), ()
        # (past ts, past global covariates, past local invariants)
        past_data = (tf.squeeze(ts_enc), tf.transpose(time_features_enc[0,:, :], perm=(1,0)), tf.zeros((1, source_seq_len)))
        # (nx, H), () 
        future_features = (tf.transpose(time_features_dec[0,:,:], perm=(1,0)), tf.zeros((1,target_seq_len)))
        # (B,)
        tsidx = tf.squeeze(cat_local_invariant_enc) # (B, )
        # (B, H)
        targets = tf.squeeze(ts_dec)

        loss = train_step((past_data, future_features, tsidx), targets)
        
    # print loss every epoch
    print(f"Epoch {epoch+1}/{MAX_EPOCHS} training loss: {loss:.4f}, MAE: {train_metrics[0].result():.4f}")
    
    # reset train metrics
    for metric in train_metrics:
        metric.reset_states()
    
    # run validation loop
    # how to run validaiton loop without batching?
    
    for val_batch in val_ds:
        enc, dec = val_batch
        (ts_enc, num_global_enc, cat_global_enc, 
         num_local_variant_enc, cat_local_variant_enc, 
         num_local_invariant_enc, cat_local_invariant_enc, time_features_enc) = enc
        (ts_dec, num_global_dec, cat_global_dec, 
         num_local_variant_dec, cat_local_variant_dec, 
         num_local_invariant_dec, cat_local_invariant_dec, time_features_dec) = dec

        # Note the orginal paper uses constant 0 for categorical features if not specified.
        # (B, L), (nx, L), ()
        # (past ts, past global covariates, past local invariants)
        past_data = (tf.squeeze(ts_enc), tf.transpose(time_features_enc[0,:, :], perm=(1,0)), tf.zeros((1, source_seq_len)))
        # (nx, H), () 
        future_features = (tf.transpose(time_features_dec[0,:,:], perm=(1,0)), tf.zeros((1,target_seq_len)))
        # (B,)
        tsidx = tf.squeeze(cat_local_invariant_enc) # (B, )
        # (B, H)
        targets = tf.squeeze(ts_dec)

        loss_val = train_step((past_data, future_features, tsidx), targets)
        
        # print loss every epoch
    print(f"Epoch {epoch+1}/{MAX_EPOCHS} validation loss: {loss_val:.4f}, MAE: {val_metrics[0].result():.4f}")
    
    # reset val metrics
    for metric in val_metrics:
        metric.reset_states()
    
    ## early stopping
    wait += 1
    if loss_val < best_val_loss:
        best_val_loss = loss_val
        wait = 0
        model.save_weights("tide.h5")
    if wait > patience:
        print('early stopping...')
        break


2023-09-14 10:11:43.000902: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-09-14 10:11:45.093152: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-14 10:11:45.240622: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-09-14 10:11:45.360976: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fbc59d274e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-14 10:11:45.361044: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-09-14 10:11:45.37218

Epoch 1/100 training loss: 0.4462, MAE: 0.5265


2023-09-14 10:13:57.975970: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/100 validation loss: 0.5423, MAE: 0.0000


KeyboardInterrupt: 

## Test

In [ ]:
# run test loop     
for test_batch in test_ds:
    enc, dec = test_batch
    (ts_enc, num_global_enc, cat_global_enc, 
        num_local_variant_enc, cat_local_variant_enc, 
        num_local_invariant_enc, cat_local_invariant_enc, time_features_enc) = enc
    (ts_dec, num_global_dec, cat_global_dec, 
        num_local_variant_dec, cat_local_variant_dec, 
        num_local_invariant_dec, cat_local_invariant_dec, time_features_dec) = dec
        
    # Note the orginal paper uses constant 0 for categorical features if not specified.
    # (B, L), (nx, L), ()
    # (past ts, past global covariates, past local invariants)
    past_data = (tf.squeeze(ts_enc), tf.transpose(time_features_enc[0,:, :], perm=(1,0)), tf.zeros((1, source_seq_len)))
    # (nx, H), () 
    future_features = (tf.transpose(time_features_dec[0,:,:], perm=(1,0)), tf.zeros((1,target_seq_len)))
    # (B,)
    tsidx = tf.squeeze(cat_local_invariant_enc) # (B, )
    # (B, H)
    targets = tf.squeeze(ts_dec)

    test_step((past_data, future_features, tsidx), targets)
    
# print loss every epoch
print(f"Test loss MSE: {test_metrics[0].result():.4f}, MAE: {test_metrics[1].result():.4f}")
    
# reset val metrics
for metric in test_metrics:
    metric.reset_states()

print(model.summary())